Import packages

In [2]:
#https://spark.apache.org/docs/latest/api/python/pyspark.sql.html
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

Read epa data in from Azure Storage, cluster must be configured to access WASB

In [4]:
epaMpg = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/epaMpg.csv', header=True, inferSchema=True)
epaMpg.show()

+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
_c0|Represented.Test.Veh.Make| Model|Vehicle.Type|HorsePower|Cylinders|Tested.Transmission.Type.Code|Tested.Transmission.Type|Gears|Drive.System.Code|Weight|AxleRatio|Test.Procedure.Cd|Test.Procedure.Description|Test.Fuel.Type.Cd|Test.Fuel.Type.Description|FuelEcon|
+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
 1| Aston Martin| Rapide S| Car| 552| 12| SA| Semi-Automatic| 8| R| 4750| 2.73| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 17.3|
 2| Aston Martin| Vanquish| Car| 568| 12| SA| Semi-Automatic| 8| R| 4500| 2.73| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 16.5|
 3| BENTLEY| Continental GT| Car| 616| 12| SA| Semi-Automatic| 8| F| 6000| 2.85| 90| US06| 61| Tier 2 Cert Gasoline| 17.4|
 4| BENTLEY| Continental GT| Car| 616| 12| SA| Semi-Automatic| 8| F| 6000| 2.85| 11| Cold CO| 27| Cold CO Premium (...| 13.6|
 5| BMW| 230i Convertible| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 3| HWFE| 61| Tier 2 Cert Gasoline| 45.8|
 6| BMW| 230i Coupe| Car| 248| 4| M| Manual| 6| R| 3625| 3.91| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 26.4|
 7| BMW| 230i Coupe| Car| 248| 4| SA| Semi-Automatic| 8| R| 3625| 2.81| 3| HWFE| 61| Tier 2 Cert Gasoline| 50.6|
 8| BMW|230i xDrive Conve...| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.6|
 9| BMW| 230i xDrive Coupe| Car| 248| 4| SA| Semi-Automatic| 8| R| 3750| 2.81| 21| Federal fuel 2-da...| 61| Tier 2 Cert Gasoline| 30.3|
 10| BMW| 320i| Both| 181| 4| A| Automatic| 8| R| 3625| 3.2| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 30.4|
 11| BMW| 320i| Both| 180| 4| M| Manual| 6| R| 3625| 3.08| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.5|
 12| BMW| 320i xDrive| Both| 181| 4| A| Automatic| 8| R| 3750| 3.15| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.9|
 13| BMW| 328d| Car| 183| 4| SA| Semi-Automatic| 8| R| 3750| 2.81| 2| CVS 75 and later ...| 19| Federal Cert Dies...| 42.2|
 14| BMW| 328d xDrive| Car| 181| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 2| CVS 75 and later ...| 19| Federal Cert Dies...| 41.2|
 15| BMW|328d xDrive Sport...| Car| 181| 4| SA| Semi-Automatic| 8| R| 4250| 2.81| 3| HWFE| 19| Federal Cert Dies...| 57.2|
 16| BMW| 330e| Car| 180| 4| SA| Semi-Automatic| 8| R| 4250| 2.93| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 35.6|
 17| BMW| 330i| Car| 248| 4| SA| Semi-Automatic| 8| R| 3875| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 30.8|
 18| BMW| 330i| Car| 248| 4| M| Manual| 6| R| 3750| 3.91| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 26.7|
 19| BMW| 330i xDrive| Car| 248| 4| SA| Semi-Automatic| 8| R| 4000| 2.81| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.7|
 20| BMW|330i xDrive Gran ...| Car| 248| 4| SA| Semi-Automatic| 8| R| 4250| 2.93| 31| Federal fuel 3-da...| 61| Tier 2 Cert Gasoline| 29.3|
+---+-------------------------+--------------------+------------+----------+---------+-----------------------------+------------------------+-----+-----------------+------+---------+-----------------+--------------------------+-----------------+--------------------------+--------+
only showing top 20 rows

In [5]:
type(epaMpg)

Out[ 28 ]: pyspark.sql.dataframe.DataFrame

In [6]:
epaMpg.printSchema()

root
-- _c0: integer (nullable = true)
-- Represented.Test.Veh.Make: string (nullable = true)
-- Model: string (nullable = true)
-- Vehicle.Type: string (nullable = true)
-- HorsePower: integer (nullable = true)
-- Cylinders: integer (nullable = true)
-- Tested.Transmission.Type.Code: string (nullable = true)
-- Tested.Transmission.Type: string (nullable = true)
-- Gears: integer (nullable = true)
-- Drive.System.Code: string (nullable = true)
-- Weight: integer (nullable = true)
-- AxleRatio: double (nullable = true)
-- Test.Procedure.Cd: integer (nullable = true)
-- Test.Procedure.Description: string (nullable = true)
-- Test.Fuel.Type.Cd: integer (nullable = true)
-- Test.Fuel.Type.Description: string (nullable = true)
-- FuelEcon: double (nullable = true)

In [7]:
epaMpg.columns

Out[ 30 ]: 
['_c0',
 'Represented.Test.Veh.Make',
 'Model',
 'Vehicle.Type',
 'HorsePower',
 'Cylinders',
 'Tested.Transmission.Type.Code',
 'Tested.Transmission.Type',
 'Gears',
 'Drive.System.Code',
 'Weight',
 'AxleRatio',
 'Test.Procedure.Cd',
 'Test.Procedure.Description',
 'Test.Fuel.Type.Cd',
 'Test.Fuel.Type.Description',
 'FuelEcon']

In [8]:
epaMpg.corr('FuelEcon','Weight')

Out[ 31 ]: -0.5713160000548484

In [9]:
epaMpg.select("`Represented.Test.Veh.Make`").distinct().count()

Out[ 32 ]: 45

In [10]:
epaMpg.filter(epaMpg.FuelEcon > 50).count()

In [11]:
epaMpg.first()

In [12]:

epaMpg.groupBy("`Represented.Test.Veh.Make`").agg({'FuelEcon': 'mean'}).collect()

In [13]:
%sql 
select `Represented.Test.Veh.Make`, avg(FuelEcon) as AvgEcon
from epaMpg
group by `Represented.Test.Veh.Make`
order by AvgEcon
limit 10 

Represented.Test.Veh.Make,AvgEcon
BUGATTI,9.8
LAMBORGHINI,12.87
Rolls Royce,14.479999999999999
BENTLEY,14.533333333333335
Aston Martin,16.9
Ferrari,17.139999999999997
MASERATI,18.6
FORD,19.2
Karma,19.2
Dodge,19.358823529411765


In [14]:
epaMpg.count()

In [15]:
epaMpg.count() - epaMpg.dropDuplicates().count()

In [16]:

carModels = epaMpg.select("`Model`", "`Represented.Test.Veh.Make`")
carModels.show()



+--------------------+-------------------------+
 Model|Represented.Test.Veh.Make|
+--------------------+-------------------------+
 Rapide S| Aston Martin|
 Vanquish| Aston Martin|
 Continental GT| BENTLEY|
 Continental GT| BENTLEY|
 230i Convertible| BMW|
 230i Coupe| BMW|
 230i Coupe| BMW|
230i xDrive Conve...| BMW|
 230i xDrive Coupe| BMW|
 320i| BMW|
 320i| BMW|
 320i xDrive| BMW|
 328d| BMW|
 328d xDrive| BMW|
328d xDrive Sport...| BMW|
 330e| BMW|
 330i| BMW|
 330i| BMW|
 330i xDrive| BMW|
330i xDrive Gran ...| BMW|
+--------------------+-------------------------+
only showing top 20 rows

Register a dataframe as a globalb temp table

In [18]:


carModels.createGlobalTempView("globalCarModels")




In [19]:

spark.sql("SELECT * FROM global_temp.globalCarModels").show()


In [20]:
#epaMpg.describe().show()

epaMpg.select("FuelEcon", "HorsePower", "Weight", "Gears","`Tested.Transmission.Type`").describe().show()


In [21]:
spark.sql("select * from epaMpg limit 10").show()

In [22]:
%sql 
select `Represented.Test.Veh.Make`,Model,HorsePower,FuelEcon
from epaMpg

order by HorsePower 

In [23]:
%sql 
select HorsePower,FuelEcon from epaMpg

In [24]:
epaMpg.corr("HorsePower","FuelEcon")

In [25]:
%sql 
select Fuelecon,Weight from epaMpg

In [26]:
%sql 
select HorsePower,Weight from epaMpg

In [27]:
%sql 
select `Represented.Test.Veh.Make`,Model,HorsePower,FuelEcon
from epaMpg
WHERE HorsePower > 400 and FuelEcon > 30
order by FuelEcon 

In [28]:
epaData = epaMpg.select("FuelEcon", "HorsePower", "Weight", "Gears")

In [29]:


splits = epaData.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 728 Testing Rows: 306

In [30]:
type(splits)

Out[ 38 ]: list

In [31]:
train.show()

+--------+----------+------+-----+
FuelEcon|HorsePower|Weight|Gears|
+--------+----------+------+-----+
 10.0| 740| 4250| 7|
 10.4| 1500| 4750| 7|
 11.3| 381| 6500| 6|
 11.4| 381| 6000| 6|
 11.8| 740| 4250| 7|
 11.9| 740| 4000| 7|
 12.0| 700| 4250| 7|
 12.5| 700| 6000| 8|
 12.6| 700| 4000| 7|
 12.7| 275| 6000| 6|
 12.8| 520| 6000| 8|
 12.9| 560| 4750| 8|
 13.1| 345| 6500| 6|
 13.3| 261| 4750| 5|
 13.4| 261| 4750| 5|
 13.5| 610| 4250| 7|
 13.5| 630| 4000| 7|
 13.6| 616| 6000| 8|
 13.7| 707| 4750| 6|
 13.8| 530| 6500| 8|
+--------+----------+------+-----+
only showing top 20 rows

In [32]:
#"FuelEcon", "HorsePower", "Weight", "Gears","`Tested.Transmission.Type`"
# Now we'll assemble a vector of all the numeric feature columns (other than ArrDelay, which we wouldn't have for enroute flights)
assembler = VectorAssembler(inputCols = ["FuelEcon", "HorsePower", "Weight", "Gears"], outputCol="features")

type(assembler)

Out[ 40 ]: pyspark.ml.feature.VectorAssembler

In [33]:
training = assembler.transform(train).select(col("features"), (col("FuelEcon").cast("Int").alias("label")))
training.show()

+--------------------+-----+
 features|label|
+--------------------+-----+
[10.0,740.0,4250....| 10|
[10.4,1500.0,4750...| 10|
[11.3,381.0,6500....| 11|
[11.4,381.0,6000....| 11|
[11.8,740.0,4250....| 11|
[11.9,740.0,4000....| 11|
[12.0,700.0,4250....| 12|
[12.5,700.0,6000....| 12|
[12.6,700.0,4000....| 12|
[12.7,275.0,6000....| 12|
[12.8,520.0,6000....| 12|
[12.9,560.0,4750....| 12|
[13.1,345.0,6500....| 13|
[13.3,261.0,4750....| 13|
[13.4,261.0,4750....| 13|
[13.5,610.0,4250....| 13|
[13.5,630.0,4000....| 13|
[13.6,616.0,6000....| 13|
[13.7,707.0,4750....| 13|
[13.8,530.0,6500....| 13|
+--------------------+-----+
only showing top 20 rows

In [34]:
lr = LinearRegression(labelCol="label",featuresCol="features", maxIter=10, regParam=0.3)
model = lr.fit(training)
print ("Model trained!")

Model trained!

In [35]:
# Generate the features vector and label
testing = assembler.transform(test).select(col("features"), (col("FuelEcon")).cast("Int").alias("trueLabel"))
testing.show()

+--------------------+---------+
 features|trueLabel|
+--------------------+---------+
[9.2,1500.0,4750....| 9|
[11.0,700.0,6000....| 11|
[11.2,700.0,4250....| 11|
[11.7,345.0,6500....| 11|
[13.5,285.0,6000....| 13|
[13.5,621.0,4500....| 13|
[13.7,707.0,6000....| 13|
[13.8,680.0,4750....| 13|
[14.0,261.0,4500....| 14|
[14.1,610.0,3750....| 14|
[14.2,345.0,5250....| 14|
[14.3,563.0,6000....| 14|
[14.6,785.0,4250....| 14|
[14.8,345.0,6000....| 14|
[15.0,395.0,5250....| 15|
[15.0,610.0,4000....| 15|
[15.1,624.0,5500....| 15|
[15.3,650.0,4250....| 15|
[15.8,383.0,6500....| 15|
[16.2,390.0,6500....| 16|
+--------------------+---------+
only showing top 20 rows

In [36]:
prediction = model.transform(testing)
predicted = prediction.select("features", "prediction", "trueLabel")
predicted.show()

+--------------------+------------------+---------+
 features| prediction|trueLabel|
+--------------------+------------------+---------+
[9.2,1500.0,4750....| 7.670476333029644| 9|
[11.0,700.0,6000....| 10.450748084713| 11|
[11.2,700.0,4250....|11.023120356636493| 11|
[11.7,345.0,6500....|11.624527895622574| 11|
[13.5,285.0,6000....|13.526379287150366| 13|
[13.5,621.0,4500....|13.266406176460993| 13|
[13.7,707.0,6000....|12.976977936255064| 13|
[13.8,680.0,4750....| 13.39407097600868| 13|
[14.0,261.0,4500....|14.368052456107451| 14|
[14.1,610.0,3750....|14.010236840947822| 14|
[14.2,345.0,5250....|14.243057846094986| 14|
[14.3,563.0,6000....| 13.78674693633876| 14|
[14.6,785.0,4250....| 14.07429179283605| 14|
[14.8,345.0,6000....|14.646076037153911| 14|
[15.0,395.0,5250....|14.875209530446469| 15|
[15.0,610.0,4000....|14.802621625081587| 15|
[15.1,624.0,5500....|14.542051949423719| 15|
[15.3,650.0,4250....| 14.93677621848297| 15|
[15.8,383.0,6500....| 15.39665967403972| 15|
[16.2,390.0,6500....|15.769384705578366| 16|
+--------------------+------------------+---------+
only showing top 20 rows

In [37]:
predicted.createOrReplaceTempView("regressionPredictions")

In [38]:
%sql
SELECT trueLabel, prediction FROM regressionPredictions

trueLabel,prediction
9,7.670476333029644
11,10.450748084713
11,11.023120356636493
11,11.624527895622574
13,13.526379287150366
13,13.266406176460993
13,12.976977936255064
13,13.39407097600868
14,14.368052456107451
14,14.010236840947822


In [39]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="trueLabel", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predicted)
print ("Root Mean Square Error (RMSE):", rmse)

Root Mean Square Error (RMSE): 0.557612781802538